In [1]:
# Importing necessary libaries

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)
import re

In [2]:
df_stations=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/weather_stations_CA.csv")




In [3]:
df_stations.drop(["Unnamed: 0", "index"],axis=1, inplace= True)

In [4]:
df_stations

,station_id,latitude,longitude,datacoverage,mindate,maxdate,min_year,max_year,data_period
0,COOP:024299,32.88333,-114.46667,0.9996,1948-01-01,1949-08-01,1948,1949,1
1,COOP:026250,34.15470,-114.29080,0.9755,1931-01-01,2015-11-01,1931,2015,84
2,COOP:029662,32.73333,-114.61667,0.9906,1948-01-01,1974-04-01,1948,1974,26
3,COOP:040006,39.03333,-122.43333,0.8618,1960-08-01,1962-12-01,1960,1962,2
4,COOP:040010,38.21770,-121.20130,0.9469,1994-01-01,2015-11-01,1994,2015,21
...,...,...,...,...,...,...,...,...,...
4423,WBAN:93242,36.98778,-120.11056,1.0000,2005-01-01,2021-03-04,2005,2021,16
4424,WBAN:93243,37.23810,-120.88250,1.0000,2004-03-25,2021-03-04,2004,2021,17
4425,WBAN:93244,34.60694,-120.07556,1.0000,2005-08-31,2021-03-04,2005,2021,16
4426,WBAN:93245,38.32080,-123.07470,1.0000,2008-06-14,2021-03-04,2008,2021,13


In [5]:
df_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4428 entries, 0 to 4427
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_id    4428 non-null   object 
 1   latitude      4428 non-null   float64
 2   longitude     4428 non-null   float64
 3   datacoverage  4428 non-null   float64
 4   mindate       4428 non-null   object 
 5   maxdate       4428 non-null   object 
 6   min_year      4428 non-null   int64  
 7   max_year      4428 non-null   int64  
 8   data_period   4428 non-null   int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 311.5+ KB


The basic functions of the H3 library are for indexing locations, which transforms latitude and longitude pairs to a 64-bit H3 index, identifying a grid cell. The function geoToH3 takes a latitude, longitude, and resolution (between 0 and 15, with 0 being coarsest and 15 being finest), and returns an index. h3ToGeo and h3ToGeoBoundary are the inverse of this function, providing the center coordinates and outline of the grid cell specified by the H3 index, respectively.

Neighboring hexagons have the useful property of approximating circles using the grid system. The kRing function provides grid cells within grid distance k of an origin index.

Dense hexagons representing California stand in stark contrast to compact hexagons representing the state, requiring many fewer hexagons to represent the same area.

https://eng.uber.com/h3/

In [6]:
#!pip install h3
#!pip install shapely
#!pip install folium
#!pip install os_sys 

import h3
from shapely.geometry import Polygon, Point
import shapely.wkt
import math
import folium
import webbrowser #part of python standard libary
import os
from folium import Map

Table of Cell Areas for H3 Resolutions: https://h3geo.org/docs/core-library/restable

In [7]:
#FUNCTION: Get H3 index
#resolution goes from 1-16 (1 is the whole world)

def hex_3_point(lat, long, resolution = 9):
    """FUNCTION: Get H3 index, resolution goes from 1-16 (1 is the whole world)"""
    index = h3.geo_to_h3(lat=lat, lng = long, resolution = resolution)
    return index

In [8]:
index=hex_3_point(df_stations["latitude"][0], df_stations["longitude"][0])
index

'89485b5511bffff'

In [9]:
#FUNCTION: returns a set of H3 attributes, given a H3 index

def output_h3_id_attributes(h3_id):
    """#FUNCTION: returns a set of H3 attributes, given a H3 index"""
    return {
        "co_ordinates" : h3.h3_to_geo(h3_id), 
        "geo_boundary" : Polygon(h3.h3_to_geo_boundary(h3_id, geo_json=True)).wkt, 
        "parent" : h3.h3_to_parent(h3_id), 
        "children" : h3.h3_to_children(h3_id)
    }

In [10]:
test_dic=output_h3_id_attributes(index)

In [11]:
test_dic

{'co_ordinates': (32.88264517584788, -114.46809124356123),
 'geo_boundary': 'POLYGON ((-114.466956063367 32.88096811193556, -114.4657704672515 32.88264873594603, -114.466905650706 32.88432579639124, -114.4692264532656 32.88432220963172, -114.4704120038574 32.88264157291489, -114.4692767974161 32.88096453566389, -114.466956063367 32.88096811193556))',
 'parent': '88485b5511fffff',
 'children': {'8a485b551187fff',
  '8a485b55118ffff',
  '8a485b551197fff',
  '8a485b55119ffff',
  '8a485b5511a7fff',
  '8a485b5511affff',
  '8a485b5511b7fff'}}

Another useful function in H3 is its ability to quickly return an index’s k-nearest neighbors. This means that for k =1, a hexagon’s 1st degree neighbors are returned, and for k = 2, it is second degree neighbors (or neighbors of neighbors) and so on.

https://towardsdatascience.com/uber-h3-for-data-analysis-with-python-1e54acdcc908

In [12]:
h3_id = index
h3.k_ring(h3_id,1)
#h3.k_ring(h3_id,2)
#h3.k_ring(h3_id,10)

{'89485b55103ffff',
 '89485b5510bffff',
 '89485b55113ffff',
 '89485b5511bffff',
 '89485b55157ffff',
 '89485b551c7ffff',
 '89485b551cfffff'}

https://github.com/joaofig/uk-accidents/blob/master/uk-accident-h3.ipynb

In [13]:
def create_map(clusters,df):
    # Create the map object
    map = Map(tiles="cartodbpositron", 
          attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        tooltip = "{0} weather stations".format(cluster['count'])
        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#1874CD', 
                                               fill_color='#1874CD', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map)

    # Determine the map bounding box
    max_lat = df.latitude.max()
    min_lat = df.latitude.min()
    max_lon = df.longitude.max()
    min_lon = df.longitude.min()
    
    # Fit the map to the bounds
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map


In [14]:
def show_map(map, file_name):
    """The show_map function saves the HTML generated by the map into a file and then opens a new browser tab with its contents."""
    map.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)

In [15]:
#radians for longitude and latitude

df_stations['rad_lng'] = np.radians(df_stations['longitude'].to_numpy())
df_stations['rad_lat'] = np.radians(df_stations['latitude'].to_numpy())

In [16]:
from sklearn.cluster import DBSCAN

eps_in_meters = 2000.0 #in meters
num_samples = 5

earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

df_stations['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine').fit_predict(df_stations[['rad_lng', 'rad_lng']])



In [17]:
df_stations
#Select only the locations that belong to a DBSCAN-generated cluster. Clusters marked with -1 are noise.

,station_id,latitude,longitude,datacoverage,mindate,maxdate,min_year,max_year,data_period,rad_lng,rad_lat,cluster
0,COOP:024299,32.88333,-114.46667,0.9996,1948-01-01,1949-08-01,1948,1949,1,-1.997820,0.573922,-1
1,COOP:026250,34.15470,-114.29080,0.9755,1931-01-01,2015-11-01,1931,2015,84,-1.994751,0.596112,-1
2,COOP:029662,32.73333,-114.61667,0.9906,1948-01-01,1974-04-01,1948,1974,26,-2.000438,0.571304,0
3,COOP:040006,39.03333,-122.43333,0.8618,1960-08-01,1962-12-01,1960,1962,2,-2.136865,0.681260,1
4,COOP:040010,38.21770,-121.20130,0.9469,1994-01-01,2015-11-01,1994,2015,21,-2.115362,0.667025,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4423,WBAN:93242,36.98778,-120.11056,1.0000,2005-01-01,2021-03-04,2005,2021,16,-2.096325,0.645559,7
4424,WBAN:93243,37.23810,-120.88250,1.0000,2004-03-25,2021-03-04,2004,2021,17,-2.109798,0.649927,1
4425,WBAN:93244,34.60694,-120.07556,1.0000,2005-08-31,2021-03-04,2005,2021,16,-2.095714,0.604005,7
4426,WBAN:93245,38.32080,-123.07470,1.0000,2008-06-14,2021-03-04,2008,2021,13,-2.148059,0.668824,1


In [419]:
#df_stations["cluster"].unique()

In [420]:
#df_stations["cluster"].value_counts()

In [18]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['latitude'], row['longitude'], h3_level)

In [19]:
h3_level=4

In [20]:
df_stations['h3_level_4'] = df_stations.apply(lat_lng_to_h3, axis=1)

In [68]:
h3_level=5

In [69]:
df_stations['h3_level_5'] = df_stations.apply(lat_lng_to_h3, axis=1)

In [23]:
df_stations

,station_id,latitude,longitude,datacoverage,mindate,maxdate,min_year,max_year,data_period,rad_lng,rad_lat,cluster,h3_level_4,h3_level_3
0,COOP:024299,32.88333,-114.46667,0.9996,1948-01-01,1949-08-01,1948,1949,1,-1.997820,0.573922,-1,84485b5ffffffff,83485bfffffffff
1,COOP:026250,34.15470,-114.29080,0.9755,1931-01-01,2015-11-01,1931,2015,84,-1.994751,0.596112,-1,8429b55ffffffff,8329b5fffffffff
2,COOP:029662,32.73333,-114.61667,0.9906,1948-01-01,1974-04-01,1948,1974,26,-2.000438,0.571304,0,84485bdffffffff,83485bfffffffff
3,COOP:040006,39.03333,-122.43333,0.8618,1960-08-01,1962-12-01,1960,1962,2,-2.136865,0.681260,1,8428339ffffffff,832833fffffffff
4,COOP:040010,38.21770,-121.20130,0.9469,1994-01-01,2015-11-01,1994,2015,21,-2.115362,0.667025,1,8428329ffffffff,832836fffffffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4423,WBAN:93242,36.98778,-120.11056,1.0000,2005-01-01,2021-03-04,2005,2021,16,-2.096325,0.645559,7,8429abbffffffff,8329abfffffffff
4424,WBAN:93243,37.23810,-120.88250,1.0000,2004-03-25,2021-03-04,2004,2021,17,-2.109798,0.649927,1,8428369ffffffff,832836fffffffff
4425,WBAN:93244,34.60694,-120.07556,1.0000,2005-08-31,2021-03-04,2005,2021,16,-2.095714,0.604005,7,8429ac9ffffffff,8329acfffffffff
4426,WBAN:93245,38.32080,-123.07470,1.0000,2008-06-14,2021-03-04,2008,2021,13,-2.148059,0.668824,1,842831dffffffff,832831fffffffff


In [70]:
len(list(df_stations["station_id"].unique()))

4427

In [432]:
len(list(df_stations['h3_level_3'].unique()))

51

In [426]:
#df_station_with_noise=df_stations
#deleting cluster == -1 (sum=83;)
#df_stations = df_stations[df_stations.cluster != -1]

In [427]:
len(list(df_stations['h3_level_4'].unique()))

247

In [25]:
#df_stations.to_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/CA_weather_stations_clustered_4.csv")

print("Sucessfully saved")

Sucessfully saved


233 vs 1888 (h3_level=4 vs h3_level=5)

In [26]:
clusters_weather = dict()

for index, row in df_stations.iterrows():
    key = row['h3_level_4']
    #print(key)
    if key in clusters_weather:
        clusters_weather[key]['count'] += 1
    else:
        clusters_weather[key] = {"count": 1,
                         "geom": h3.h3_to_geo_boundary(key)}

In [27]:
len(clusters_weather)
#len(list(df_stations['h3'].unique()))

247

In [28]:
map = create_map(clusters_weather,df_stations)



In [29]:
#h3_level=6
show_map(map, "map_{0}.html".format(h3_level))

# Mapping wildfires in 

In [30]:
df_wildfires=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/big_wildfires_CA.csv")

In [31]:
df_wildfires.drop(["Unnamed: 0"],axis=1, inplace=True)

In [32]:
new_columns=[]
for col in list(df_wildfires.columns):
    new_columns.append(col.lower())
    
df_wildfires.columns=new_columns
#df_wildfires

In [33]:
df_wildfires.columns

Index(['objectid', 'fod_id', 'fpa_id', 'source_system_type', 'source_system',
       'nwcg_reporting_agency', 'nwcg_reporting_unit_id',
       'nwcg_reporting_unit_name', 'source_reporting_unit',
       'source_reporting_unit_name', 'local_fire_report_id',
       'local_incident_id', 'fire_code', 'fire_name',
       'ics_209_incident_number', 'ics_209_name', 'mtbs_id', 'mtbs_fire_name',
       'complex_name', 'fire_year', 'discovery_date', 'discovery_doy',
       'discovery_time', 'stat_cause_code', 'stat_cause_descr', 'cont_date',
       'cont_doy', 'cont_time', 'fire_size', 'fire_size_class', 'latitude',
       'longitude', 'owner_code', 'owner_descr', 'state', 'county',
       'fips_code', 'fips_name', 'shape'],
      dtype='object')

In [34]:
#radians for longitude and latitude

df_wildfires['rad_lng'] = np.radians(df_wildfires['longitude'].to_numpy())
df_wildfires['rad_lat'] = np.radians(df_wildfires['latitude'].to_numpy())

In [35]:
#Density-based spatial clustering of applications with noise (DBSCAN)
from sklearn.cluster import DBSCAN

eps_in_meters = 250.0 #in meters
num_samples = 15

earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)

df_wildfires["cluster"] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine').fit_predict(df_wildfires[['rad_lng', 'rad_lng']])

#df_wildfires



In [36]:
df_wildfires["cluster"].unique()

array([  0,  -1,   1,   2,  41,   3, 238,   4,  94, 194,   5,   6,   7,
         8,  30,   9,  13,  10, 237,  11,  12,  85,  14, 211,  15,  16,
       235, 140,  17,  18,  19,  20,  21,  22,  23,  47, 145,  25,  24,
       203, 220, 173,  87, 260,  26, 157, 263,  27, 168,  28,  29,  72,
        67,  31, 243,  86,  52, 200, 242,  32, 166,  33,  34, 127, 142,
       101,  35, 100,  36,  37,  38, 259,  39, 162,  40,  51, 139, 181,
        42,  43,  83, 105,  44, 202,  75,  92,  45,  46, 141, 169,  48,
        49,  50, 118, 159, 257,  53,  54, 160,  55, 163,  56,  57,  58,
       224,  59, 209,  60, 206, 189, 182, 124,  61, 103, 135,  62,  68,
        63,  64, 143,  65, 250, 221,  66, 208,  69, 217,  70,  71, 104,
        81, 111, 270, 245,  73,  74, 201,  76,  77,  78,  79,  80,  82,
       134,  84, 213, 239, 214, 121, 226, 125,  88,  89,  90,  91,  93,
       191, 266, 265, 216, 158,  95,  96, 190, 233,  97, 262,  98, 151,
        99, 102, 106, 246, 107, 108, 136, 109, 110, 192, 112, 12

In [37]:
df_wildfires["cluster"].value_counts()

-1      6947
 38      125
 17      115
 76       88
 22       70
        ... 
 233      12
 222      12
 223      10
 194      10
 271       5
Name: cluster, Length: 274, dtype: int64

In [38]:
#df_wildfires[df_wildfires["cluster"]==0]

In [50]:
h3_level=5

In [51]:
df_wildfires['h3_level_5'] = df_wildfires.apply(lat_lng_to_h3, axis=1)

In [52]:
h3_level=3

In [53]:
df_wildfires['h3_level_3'] = df_wildfires.apply(lat_lng_to_h3, axis=1)

In [61]:
h3_level=6

In [62]:
df_wildfires['h3_level_6'] = df_wildfires.apply(lat_lng_to_h3, axis=1)

In [54]:
len(list(df_wildfires['h3_level_5'].unique()))

1293

In [55]:
df_wildfires.to_csv("/Users/carolinvogt/Becoming_Data_Analyst/final_project/wildfire_project/CA_wildfires_clustered.csv")

print("Sucessfully saved")

Sucessfully saved


In [63]:
clusters_fire = dict()

for index, row in df_wildfires.iterrows():
    key = row['h3_level_6']
    #print(key)
    if key in clusters_fire:
        clusters_fire[key]['count'] += 1
    else:
        clusters_fire[key] = {"count": 1,
                         "geom": h3.h3_to_geo_boundary(key)}


In [64]:
def create_map_fire(clusters,df):
    # Create the map object
    map = Map(tiles="cartodbpositron", 
          attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>')

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        tooltip = "{0} fire".format(cluster['count'])
        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#ff0000', 
                                               fill_color='#ff0000', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map)

    # Determine the map bounding box
    max_lat = df.latitude.max()
    min_lat = df.latitude.min()
    max_lon = df.longitude.max()
    min_lon = df.longitude.min()
    
    # Fit the map to the bounds
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map

In [65]:
map = create_map_fire(clusters_fire,df_wildfires)

In [66]:
h3_level=6

In [67]:
show_map(map, "map_{0}.html".format(h3_level))